In [1]:
import pandas as pd
from sklearn.mixture import GaussianMixture

# Load the composite score data
df = pd.read_csv("individual_scaled_scores.csv", parse_dates=["Date"])
df.set_index("Date", inplace=True)

# Smooth the scores
df["LeadingScore_Smooth"] = df["LeadingScore"].rolling(3, center=True).mean()
df["CoincidentScore_Smooth"] = df["CoincidentScore"].rolling(3, center=True).mean()

# Feature engineering
df["LeadMinusCoin"] = df["LeadingScore_Smooth"] - df["CoincidentScore_Smooth"]
df["LeadMomentum"] = df["LeadingScore_Smooth"].diff()
df["CoinMomentum"] = df["CoincidentScore_Smooth"].diff()
df.dropna(inplace=True)

# Fit GMM with 10 components (regimes)
features = ["LeadingScore_Smooth", "CoincidentScore_Smooth", "LeadMinusCoin", "LeadMomentum", "CoinMomentum"]
X = df[features]
gmm_10 = GaussianMixture(n_components=10, random_state=42)
df["Regime_10"] = gmm_10.fit_predict(X)

# Add soft probabilities
proba_10 = gmm_10.predict_proba(X)
for i in range(10):
    df[f"Prob_Regime10_{i}"] = proba_10[:, i]

# Smooth probabilities (for visualization/consistency)
proba_cols_10 = [f"Prob_Regime10_{i}" for i in range(10)]
df[proba_cols_10] = df[proba_cols_10].ewm(span=3).mean()

# Export this 10-regime experiment version
output_path = "regime_model_10_clusters.csv"
df.to_csv(output_path)

output_path


C:\Users\user\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\user\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\user\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(

'regime_model_10_clusters.csv'